## Here I will take the key features from my data set and run them through weighted classificaton models, current models are Random Forest and Logit

due to class imbalance and non-linear features my models are currently correctly identifying approx 10- 50% of the failed hard drives and mis attributing approx 5 - 25% of the working drives as broken. I will implement non-linear feature treatment and svm to address this in the coming days.

In [1]:
import pandas as pd
import sklearn
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from sklearn import svm, ensemble, linear_model, neighbors, naive_bayes

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

from sklearn.metrics import precision_score, recall_score, precision_recall_curve,f1_score, fbeta_score

In [5]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, 
                              AdaBoostClassifier, BaggingRegressor)
from sklearn import linear_model, svm, metrics, ensemble

In [6]:
from sklearn import model_selection, preprocessing

In [7]:
df = pd.read_pickle("/Users/robjohns/Data/Metis/HDfailure/Data/tot_2days_pickle.pkl")


In [8]:
df.failure.value_counts()

0    31023
1     1471
Name: failure, dtype: int64

In [9]:
df.head()

,serial_number,failure,capacity_bytes,smart_1_normalized,smart_4_raw,smart_5_raw,smart_7_raw,smart_9_normalized,smart_9_raw,smart_12_raw,...,smart_187_raw,smart_188_raw,smart_193_normalized,smart_193_raw,smart_197_raw,smart_198_raw,smart_199_raw,smart_240_raw,smart_241_raw,smart_242_raw
0,S3000FZ5,0,476093656571904,13686.0,595.0,0.0,2.817327e+10,11284.0,589167.0,595.0,...,0.0,0.0,11662.0,515961.0,0.0,0.0,0.0,1.651378e+16,1.309012e+12,1.849806e+12
1,S3000QAP,0,476093656571904,13705.0,767.0,0.0,3.112039e+10,11269.0,603537.0,767.0,...,0.0,272.0,10488.0,2873862.0,0.0,0.0,0.0,5.886970e+05,1.854754e+12,1.253666e+13
2,S3002HH0,0,476093656571904,13631.0,1208.0,0.0,1.312904e+10,11697.0,232537.0,1208.0,...,0.0,0.0,11342.0,1221007.0,0.0,0.0,0.0,2.256840e+05,9.556671e+11,3.202999e+12
3,S30034E6,0,476093656571904,13707.0,595.0,0.0,3.744539e+10,11385.0,501272.0,595.0,...,0.0,0.0,11781.0,434126.0,0.0,0.0,0.0,1.613259e+16,1.328333e+12,2.709739e+12
4,S3003GAB,0,476093656571904,13662.0,931.0,0.0,1.932952e+10,11573.0,341074.0,931.0,...,0.0,0.0,11355.0,1279592.0,0.0,0.0,0.0,3.347480e+05,1.143215e+12,5.060294e+12


In [10]:
df.count()

serial_number           32494
failure                 32494
capacity_bytes          32494
smart_1_normalized      32494
smart_4_raw             32494
smart_5_raw             32494
smart_7_raw             32494
smart_9_normalized      32494
smart_9_raw             32494
smart_12_raw            32494
smart_183_raw           32494
smart_184_raw           32494
smart_187_raw           32494
smart_188_raw           32494
smart_193_normalized    32494
smart_193_raw           32494
smart_197_raw           32494
smart_198_raw           32494
smart_199_raw           32494
smart_240_raw           32494
smart_241_raw           32494
smart_242_raw           32494
dtype: int64

In [11]:
y,X= df['failure'],df.drop(['failure','serial_number','capacity_bytes'],axis=1)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=.2, stratify=y)

In [12]:
x_test.shape

(6499, 19)

In [13]:
#scaler = preprocessing.MinMaxScaler()
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)

x_test = scaler.transform(x_test)

In [14]:
print(len(x_train), len(x_test))

25995 6499


In [15]:
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2,random_state=13)

#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [16]:
def print_metrics(model):
  
    y_pred = model.predict(x_test)

    print(metrics.classification_report(y_test, y_pred))

In [36]:
rf_weights  = {
      0: 1,
      1: 200}
RandomForest_weights = RandomForestClassifier(class_weight=rf_weights,n_estimators=100)
RandomForest_weights.fit(x_train, y_train)

print_metrics(RandomForest_weights)

              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6205
           1       0.88      0.36      0.51       294

    accuracy                           0.97      6499
   macro avg       0.92      0.68      0.75      6499
weighted avg       0.97      0.97      0.96      6499



In [39]:
rf_weights  = {
      0: 1,
      1: 200}
randomforest = RandomForestClassifier(class_weight=rf_weights,n_estimators=100)

In [40]:
randomforest.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight={0: 1, 1: 200},
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

In [41]:
randomforest.score(x_train,y_train)

1.0

In [42]:
rf_conf=confusion_matrix(y_test, randomforest.predict(x_test))
rf_conf


array([[6190,   15],
       [ 190,  104]])

In [43]:
failperc=100*(rf_conf[0])[1]/((rf_conf[0])[0]+(rf_conf[0])[1])
print('RF Incorrectly thinks', failperc,'% are going to fail')

RF Incorrectly thinks 0.24174053182917002 % are going to fail


In [44]:
caught= 100*(rf_conf[1])[1]/((rf_conf[1])[0]+(rf_conf[1])[1])
print('RF Correctly thinks', caught,'% are going to fail')

RF Correctly thinks 35.374149659863946 % are going to fail
